In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [5]:
import os 
os.getcwd() 
os.chdir('/content/gdrive/My Drive/app/flutter/ifruit')

In [6]:
!ls

fruits.zip  TrainIfruitDetector.ipynb


In [7]:
import zipfile
zip_ref = zipfile.ZipFile("fruits.zip", 'r')
zip_ref.extractall("/data")
zip_ref.close()

In [8]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [9]:
!unzip  fruits.zip

Streaming output truncated to the last 5000 lines.
  inflating: fruits-360/Training/Banana Lady Finger/191_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/192_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/193_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/194_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/195_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/196_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/197_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/198_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/199_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/19_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/1_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/20_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/21_100.jpg  
  inflating: fruits-360/Training/Banana Lady Finger/22_100.jpg  
  inflating: fruits-360/Trainin

In [10]:
import tensorflow as tf

In [11]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
base_dir = 'fruits-360/Training'
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.1,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'training'
)

print(train_generator.class_indices)

Found 44034 images belonging to 95 classes.
{'Apple Braeburn': 0, 'Apple Golden 1': 1, 'Apple Golden 2': 2, 'Apple Golden 3': 3, 'Apple Granny Smith': 4, 'Apple Red 1': 5, 'Apple Red 2': 6, 'Apple Red 3': 7, 'Apple Red Delicious': 8, 'Apple Red Yellow 1': 9, 'Apple Red Yellow 2': 10, 'Apricot': 11, 'Avocado': 12, 'Avocado ripe': 13, 'Banana': 14, 'Banana Lady Finger': 15, 'Banana Red': 16, 'Cactus fruit': 17, 'Cantaloupe 1': 18, 'Cantaloupe 2': 19, 'Carambula': 20, 'Cherry 1': 21, 'Cherry 2': 22, 'Cherry Rainier': 23, 'Cherry Wax Black': 24, 'Cherry Wax Red': 25, 'Cherry Wax Yellow': 26, 'Chestnut': 27, 'Clementine': 28, 'Cocos': 29, 'Dates': 30, 'Granadilla': 31, 'Grape Blue': 32, 'Grape Pink': 33, 'Grape White': 34, 'Grape White 2': 35, 'Grape White 3': 36, 'Grape White 4': 37, 'Grapefruit Pink': 38, 'Grapefruit White': 39, 'Guava': 40, 'Hazelnut': 41, 'Huckleberry': 42, 'Kaki': 43, 'Kiwi': 44, 'Kumquats': 45, 'Lemon': 46, 'Lemon Meyer': 47, 'Limes': 48, 'Lychee': 49, 'Mandarine': 50

In [12]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('label.txt','w')as f:
  f.write(labels)

In [13]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE, include_top=False, weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [14]:
base_model.trainable = False

In [15]:
labels

'Apple Braeburn\nApple Golden 1\nApple Golden 2\nApple Golden 3\nApple Granny Smith\nApple Red 1\nApple Red 2\nApple Red 3\nApple Red Delicious\nApple Red Yellow 1\nApple Red Yellow 2\nApricot\nAvocado\nAvocado ripe\nBanana\nBanana Lady Finger\nBanana Red\nCactus fruit\nCantaloupe 1\nCantaloupe 2\nCarambula\nCherry 1\nCherry 2\nCherry Rainier\nCherry Wax Black\nCherry Wax Red\nCherry Wax Yellow\nChestnut\nClementine\nCocos\nDates\nGranadilla\nGrape Blue\nGrape Pink\nGrape White\nGrape White 2\nGrape White 3\nGrape White 4\nGrapefruit Pink\nGrapefruit White\nGuava\nHazelnut\nHuckleberry\nKaki\nKiwi\nKumquats\nLemon\nLemon Meyer\nLimes\nLychee\nMandarine\nMango\nMangostan\nMaracuja\nMelon Piel de Sapo\nMulberry\nNectarine\nOrange\nPapaya\nPassion Fruit\nPeach\nPeach 2\nPeach Flat\nPear\nPear Abate\nPear Kaiser\nPear Monster\nPear Williams\nPepino\nPhysalis\nPhysalis with Husk\nPineapple\nPineapple Mini\nPitahaya Red\nPlum\nPlum 2\nPlum 3\nPomegranate\nPomelo Sweetie\nQuince\nRambutan\nRa

In [16]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32,3,activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(95,activation='softmax')

])

In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics = ['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 95)                3135      
Total params: 2,629,791
Trainable params: 371,807
Non-trainable params: 2,257,984
_________________________________________________________________


In [19]:
epochs = 10
history = model.fit(train_generator, epochs=epochs, steps_per_epoch=200)

Epoch 1/10
200/200 [==============================] - 147s 735ms/step - loss: 2.5061 - accuracy: 0.4027
Epoch 2/10
200/200 [==============================] - 148s 738ms/step - loss: 0.7549 - accuracy: 0.7763
Epoch 3/10
200/200 [==============================] - 148s 740ms/step - loss: 0.4568 - accuracy: 0.8601
Epoch 4/10
200/200 [==============================] - 147s 737ms/step - loss: 0.3482 - accuracy: 0.8894
Epoch 5/10
200/200 [==============================] - 148s 739ms/step - loss: 0.2976 - accuracy: 0.9048
Epoch 6/10
200/200 [==============================] - 147s 736ms/step - loss: 0.2614 - accuracy: 0.9177
Epoch 7/10
200/200 [==============================] - 147s 735ms/step - loss: 0.2287 - accuracy: 0.9261
Epoch 8/10
200/200 [==============================] - 146s 732ms/step - loss: 0.2031 - accuracy: 0.9339
Epoch 9/10
200/200 [==============================] - 149s 745ms/step - loss: 0.1936 - accuracy: 0.9366
Epoch 10/10
200/200 [==============================] - 148s 739m

In [20]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite','wb') as f:
  f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: assets


In [21]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('label.txt','w')as f:
  f.write(labels)

In [27]:
from google.colab import files
files.download('label.txt')
files.download('model.tflite')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
!pwd

/content/gdrive/My Drive/app/flutter/ifruit
